In [1]:
import xml.etree.ElementTree as ET
import sqlite3

In [2]:
conn = sqlite3.connect('trackdb.sqlite')  # make a db connection
cur = conn.cursor()  # database handle 

In [3]:
cur.executescript('''
DROP TABLE IF EXISTS Artist; 
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  name TEXT UNIQUE
);
CREATE TABLE Album(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
  artist_id INTEGER,
  title TEXT UNIQUE
);
CREATE TABLE Track(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  title TEXT UNIQUE,
  album_id INTEGER,
  len INTEGER, 
  rating INTEGER, 
  count INTEGER
);
''')

In [4]:
fname = input('Enter the file name: ')
if (len(fname) < 1): fname = 'Library.xml'

Enter the file name: Library.xml


#### <key>Track ID</key><integer>369</integer>
#### <key>Name</key><string>Another One Bites The Dust</string>
#### <key>Artist</key><string>Queen</string>

In [6]:
def lookup(d, key):
  found = False
  for child in d:
    if found : return child.text
    if child.tag == 'key' and child.text == key :
      found = True
  return None

In [9]:
stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count: ',  len(all))
for entry in all:
  if (lookup(entry, 'Track ID') is None): continue

  name = lookup(entry, 'Name')
  artist = lookup(entry, 'Artist')
  album = lookup(entry, 'Album')
  count = lookup(entry, 'Play Count')
  rating = lookup(entry, 'Rating')
  length = lookup(entry, 'Total Time')

  if name is None or artist is None or album is None:
    continue

  print(name, artist, album, count, rating, length)

  cur.execute('''Insert OR IGNORE INTO Artist(name) VALUES (?)''', (artist, ))
  cur.execute('''SELECT id FROM Artist WHERE name = ? ''', (artist, ))
  artist_id = cur.fetchone()[0]

  cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) VALUES (?, ?)''', (album, artist_id))
  cur.execute('''SELECT id FROM Album WHERE title = ?''', (album, ))
  album_id = cur.fetchone()[0]

  cur.execute('''INSERT OR REPLACE INTO Track (title, album_id, len, rating, count) VALUES (?, ?, ?, ?, ?)''', (name, album_id, length, rating, count))
  conn.commit()

Dict count:  404
Another One Bites The Dust Queen Greatest Hits 55 100 217103
Asche Zu Asche Rammstein Herzeleid 79 100 231810
Beauty School Dropout Various Grease 48 100 239960
Black Dog Led Zeppelin IV 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118
Circles Bryan Lee Blues Is 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631
Electric Funeral Black Sabbath Paranoid 44 100 293015
Fat Bottomed Girls Queen Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750
Four Sticks Led Zeppelin IV 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004
Gelle Bryan Lee Blues Is 45 60 199836
Going To California Led Zeppelin IV 100 100 215666
Grease Various Grease 42 100 205792
Hand of Doom Black Sabbath Paranoid 36 100 429609
Hells Bells AC/DC Who Made Who 82 100 312946
Hey You Pink Floyd The Wall [Disc 2] 23 100 282

In [13]:
for results in cur.execute('''
SELECT Track.title, Album.title, Artist.name FROM Track JOIN Album JOIN Artist ON Track.album_id = Album.id and Album.artist_id = Artist.id
'''):
  print(results)

('Another One Bites The Dust', 'Greatest Hits', 'Queen')
('Asche Zu Asche', 'Herzeleid', 'Rammstein')
('Beauty School Dropout', 'Grease', 'Various')
('Black Dog', 'IV', 'Led Zeppelin')
('Bring The Boys Back Home', 'The Wall [Disc 2]', 'Pink Floyd')
('Circles', 'Blues Is', 'Bryan Lee')
('Comfortably Numb', 'The Wall [Disc 2]', 'Pink Floyd')
('Crazy Little Thing Called Love', 'Greatest Hits', 'Queen')
('Electric Funeral', 'Paranoid', 'Black Sabbath')
('Fat Bottomed Girls', 'Greatest Hits', 'Queen')
('For Those About To Rock (We Salute You)', 'Who Made Who', 'AC/DC')
('Four Sticks', 'IV', 'Led Zeppelin')
('Furious Angels', 'The Matrix Reloaded', 'Rob Dougan')
('Gelle', 'Blues Is', 'Bryan Lee')
('Going To California', 'IV', 'Led Zeppelin')
('Grease', 'Grease', 'Various')
('Hand of Doom', 'Paranoid', 'Black Sabbath')
('Hells Bells', 'Who Made Who', 'AC/DC')
('Hey You', 'The Wall [Disc 2]', 'Pink Floyd')
('I Worry', 'Blues Is', 'Bryan Lee')
('Iron Man', 'Paranoid', 'Black Sabbath')
('Is Ther